### Wstęp do Uczenia Maszynowego 
##### Laboratorium 05

In [81]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import recall_score, precision_score, roc_auc_score, roc_curve

### Zadanie 1
-----
Wczytaj dane `pima.csv`.

a) Podziel dane za część treningową i testową (3:2).

b) Dopasuj model drzewa do danych treningowych.

c) Dopasuj model regresji logistycznej do danych treningowych.

d) Podaj miary: czułość, precyzja, AUC dla obu modeli na zbiorze testowym.

e) Narysuj krzywą ROC dla obu modeli na zbiorze testowym.

### Zadanie 2
-----
Eksperyment symulacyjny. Wygenerujmy dane w następujący sposób. Niech $n = 200$, $p = 5$. Wektor $\beta_0 = 1$, $\beta_1 = [2, 1, 0.5, 0.01, 0]$, $x_i \sim N_p(0, I)$ dla $i = 1,2, \dots, n$. $y_i$ pochodzi z rozkładu $Bern(p_i)$, gdzie $p_i=\frac{exp(\beta_0 + x_{i\cdot}\beta_1)}{1 + exp(\beta_0 + x_{i\cdot}\beta_1)}$.

a) Dopasuj model regresji logistycznej. Porównaj prawdziwe wartości wektora $\beta$ z wyestymowanymi.

b) Dopasuj model regresji logistycznej z regularyzacją $L2$ i współczynnikami $C = (10, 5, 2, 1, 0.5, 0.1, 0.01, 0.005)$. jak zmienieją się współczynniki wraz ze wzrostem współczynnika lambda?

c) Dopasuj model regresji logistycznej z regularyzacją $L1$ i współczynnikami $C = (10, 5, 2, 1, 0.5, 0.1, 0.01, 0.005)$. jak zmienieją się współczynniki wraz ze wzrostem współczynnika lambda?

d) Powtórz eksperyment 500 razy dla modelu logistycznego bez regularyzacji, z regularyzacją $L2$ i parametrem $C=1$ i regularyzacją $L1$ i parametrem $C=1$. Oblicz MSE.

### Zadanie 3
-----
Wpływ skalowania zmiennych. Dopasuj modele regresji logistycznej
1. z regularyzacją L2 dla danych bez skalowania i dla danych ze skalowaniem zmiennych (StandardScaler);
2. z regularyzacją L1 dla danych bez skalowania i dla danych ze skalowaniem zmiennych (StandardScaler).

Porównaj współczynniki.

In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler

# Wczytanie danych
credit = fetch_openml(name='credit-g', version=1, as_frame=True)
df = credit.frame

print(df.shape)
sel_cols = ['age', 'existing_credits', 'duration', 'credit_amount', 'savings_status', 'credit_history', 'class']
df = df[sel_cols]
df.head(10)


In [ ]:
df['class'] = (df['class'] == 'good').astype(int)

X = df.drop(columns='class')
y = df['class']

#### Regresja logistyczna z regularyzacją Elastic Net

Regresja logistyczna estymuje parametry $\boldsymbol{\beta}$ poprzez minimalizację funkcji kosztu (ujemnej log-likelihood) z dodaną karą regularyzacyjną:

$$
\mathcal{L}(\boldsymbol{\beta}) 
= -\frac{1}{n} \sum_{i=1}^{n} 
\Big[ 
y_i \log(p_i) + (1 - y_i) \log(1 - p_i) 
\Big]
+ \lambda \left(
\alpha \|\boldsymbol{\beta}\|_1 + 
\frac{1 - \alpha}{2} \|\boldsymbol{\beta}\|_2^2
\right)
$$

gdzie:

- $p_i = \frac{1}{1 + e^{-(\beta_0 + \mathbf{x}_i^\top \boldsymbol{\beta})}}$  — to prawdopodobieństwo przynależności do klasy pozytywnej,  
- $\lambda > 0$ — współczynnik regularyzacji (im większy, tym silniejsza kara),  
- $\alpha \in [0, 1]$ — współczynnik mieszający między karą $L_1$ i $L_2$:
  - dla $\alpha = 1$ → **Lasso (L1)**,
  - dla $\alpha = 0$ → **Ridge (L2)**,
  - dla $0 < \alpha < 1$ → **Elastic Net** (mieszanka obu).

---
W implementacji `scikit-learn` parametr `C` jest odwrotnością $\lambda$:

$$
C = \frac{1}{\lambda},
$$
a parametrowi $\alpha$ odpowiada `l1_ratio`.



### Zadanie 4
-----
   Dopasuj modele regresji logistycznej z regularyzacją L1 (penalty='l1') dla wartości  
     $$
     C \in \{0.001, 0.01, 0.1, 0.5, 1, 2, 5, 10, 50, 100\}.
     $$
   - Dla każdej wartości `C` oblicz:
     - wartość wskaźnika AUC na zbiorze testowym,  
     - liczbę niezerowych współczynników modelu.
   - Na podstawie wyników wybierz wartość `C`, która stanowi najlepszy kompromis między jakością predykcji a prostotą modelu.


   Następnie używając wybranego najlepszego `C`, dopasuj modele regresji logistycznej z regularyzacją Elastic Net, zmieniając parametr  
     $$
     \alpha \in \{0.0, 0.1, 0.25, 0.5, 0.75, 0.9,  1.0\}.
     $$
   - Dla każdej wartości $\alpha$ oblicz:
     - wartość AUC na zbiorze testowym,  
     - liczbę niezerowych współczynników.

In [ ]:
credit = fetch_openml(name='credit-g', version=1, as_frame=True)
df = credit.frame

# Zmienna celu: 1 = good, 0 = bad
df['class'] = (df['class'] == 'good').astype(int)

# One-hot encoding zmiennych kategorycznych
X = pd.get_dummies(df.drop(columns='class'), drop_first=True)
y = df['class']

# Podział na zbiory treningowy/testowy
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Standaryzacja danych
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)

# -----------------------------
# 2. Optymalizacja parametru C dla regularyzacji L1
# -----------------------------
C_values = np.logspace(-3, 2, 10)  # od 0.001 do 100
results_L1 = []




#### Uwaga

Wartość optymalna parametru `C` uzyskana dla regularyzacji **L1** może stanowić **dobry punkt wyjścia** do dalszej optymalizacji w modelu **Elastic Net**.  
Jednak ze względu na obecność składnika L2 w funkcji kosztu, **optymalne wartości `C` mogą się różnić** między L1 i Elastic Net.  
W praktyce warto:
- przeprowadzić osobną optymalizację obu hiperparametrów (`C`, `α`),  
- lub rozważyć **otoczenie wartości `C_max`** wyznaczonego dla L1, aby sprawdzić stabilność rozwiązania.

---